# Кратко об этом исследовании
Данные из Остинского центра животных, то есть приюта, -- с 1 октября 2013 по **март 2016**. 

### Цель

Требуется предсказать судьбу каждого животного по данным о нём сведениям. По сути, обычная задача категоризации. Классы: Adoption, Died, Euthanasia, Return to owner, Transfer. 

Все классы считаем одинаково важными все зависимости от представленности в выборке. Качество предсказаний оценивается поэтому с помощью macro-averaged F1 score.

---

**Задание**

Пользоваться в точности предложенной в этом шаблоне схемой необязательно, но в этом ноутбуке должен быть 
- внятный и чистый, 
- прокомментированный, 
- воспроизводимый (зафиксируйте все random seeds, которые возможно),
- мотивированный
**код**, который **генерирует ваше лучшее решение**.

А также пока **неформальный отчёт** о проделанной работе.


### Методы

Я процессе исследования я пытался использовать разные модели: Логистическую регрессию, решающие деревья, случайный лес, бустинги.

Я пробовал разделять датасет отдельно на кошек и собак, но большого результата это не дало.

Были попытки предобрабатывать данные с помощью SVD, но также не получил какого-то улучшения.


### Результаты

Лучшие результаты у меня показали решающие деревья и на самом деле очень удивительно, что эти результаты достигнуты при достаточно большом количестве деревьев.

Дальше в ноутбуке описано, как я предобрабатываю данные и какие параметры использую.

---
---
---

## Конфиги и константы



In [22]:
OUTCOME2LABEL = {"Adoption": 0,
                 "Transfer": 1,
                 "Return_to_owner": 2,
                 "Euthanasia": 3,
                 "Died": 4
                 }
LABEL2OUTCOME = {v: k for k, v in OUTCOME2LABEL.items()}

score = "f1_macro"

FOLD_K = 4


## Библиотеки

In [23]:
import pandas as pd
import numpy as np
import eli5

from scipy import sparse
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

In [24]:
df_train = pd.read_csv("train.csv", encoding="utf-8")
df_test = pd.read_csv("test.csv", encoding="utf-8")
df_train.head(5)

,Name,SexuponOutcome,AnimalType,AgeuponOutcome,Breed,Color,DateTime,Outcome,ID
0,Socks,Neutered Male,Cat,2 months,Domestic Shorthair Mix,Black/White,2014-06-11 14:36:00,0,0
1,Vera,Intact Female,Cat,1 month,Domestic Shorthair Mix,Tortie/White,2014-07-18 08:10:00,3,1
2,Biscuit,Neutered Male,Dog,3 months,Chihuahua Shorthair Mix,Yellow,2016-01-02 17:28:00,2,2
3,Kitten,Spayed Female,Cat,2 years,Domestic Shorthair Mix,Calico,2014-02-19 17:27:00,0,3
4,NaN,Neutered Male,Cat,2 months,Domestic Shorthair Mix,Orange Tabby,2014-07-21 17:34:00,0,4


### Подготовка признаков

#### Даты

Даты преобразуем в количество дней от даты поступления.

In [25]:
def date2numbers(date: str):
    year, month, date = map(int, date.split(' ')[0].split('-')[:3])
    return (2016 - year) * 24 * 30 + month * 30 + date


def pandas_dates2number(date_series: pd.Series):
    return date_series.apply(date2numbers)


pandas_dates2number(df_train["DateTime"])


0        1631
1        1668
2          32
3        1519
4        1671
         ... 
18705     854
18706    2488
18707    1805
18708    1649
18709    2544
Name: DateTime, Length: 18710, dtype: int64

#### Возраст

Возраст преобразуем просто в количество дней. Отсутствующие значение меняем на средние.

In [26]:
def pandas_ages2number(date_series: pd.Series):
    def transform_age_line(age):
        if type(age) is not str:
            return float('nan')
        num, time = age.split()
        return int(num) * (("month" in time) + ("year" in time) * 12)
    numbers_df = date_series.apply(transform_age_line)
    mean = numbers_df.mean()
    numbers_df = numbers_df.fillna(value=mean)
    return numbers_df


pandas_ages2number(pd.Series([float('nan'), "2 years", "3 month"]))


0    13.5
1    24.0
2     3.0
dtype: float64

#### Цвета и породы

Повторяющихся категорий много, так что можно закодировать их и так, однако **некоторые из них можно и растащить на части**


In [27]:
df_train["Color"].value_counts()


Black/White                1955
Black                      1594
Brown Tabby                1122
Brown Tabby/White           680
White                       649
                           ... 
Tricolor/Calico               1
Blue Tabby/Orange             1
Black Smoke/Brown Tabby       1
Lynx Point/Brown Tabby        1
Liver Tick/White              1
Name: Color, Length: 326, dtype: int64

#####  CountVectorizer


In [28]:
vectorizer = CountVectorizer()
vectorized_color = vectorizer.fit_transform(df_train["Color"])

print(vectorizer.vocabulary_)

vectorized_color


{'black': 2, 'white': 34, 'tortie': 32, 'yellow': 35, 'calico': 7, 'orange': 18, 'tabby': 27, 'tan': 28, 'buff': 6, 'brown': 5, 'brindle': 4, 'chocolate': 8, 'blue': 3, 'tricolor': 33, 'red': 21, 'torbie': 31, 'sable': 23, 'cream': 9, 'merle': 17, 'gray': 13, 'lynx': 16, 'point': 20, 'flame': 11, 'lilac': 14, 'gold': 12, 'seal': 24, 'silver': 25, 'smoke': 26, 'fawn': 10, 'tick': 29, 'apricot': 1, 'liver': 15, 'tiger': 30, 'agouti': 0, 'pink': 19, 'ruddy': 22}


<18710x36 sparse matrix of type '<class 'numpy.int64'>'
	with 33576 stored elements in Compressed Sparse Row format>

## Breed 
Если мы посмотрим на количества определенных пород, то увидим, что у нас несколько пород встречаются довольно часто, а остальные все менее редко.
Также заметим, что у нас некоторые породы встречаются всего 1 раз, поэтому мы можем ожидать, что на тесте мы увидим новые породы.


In [29]:
df_train.Breed.value_counts()

Domestic Shorthair Mix            6153
Pit Bull Mix                      1318
Chihuahua Shorthair Mix           1225
Labrador Retriever Mix             984
Domestic Medium Hair Mix           590
                                  ... 
Cairn Terrier/Miniature Poodle       1
Jack Russell Terrier/Pointer         1
Whippet/Australian Kelpie            1
Miniature Schnauzer/Shih Tzu         1
Great Dane/Pit Bull                  1
Name: Breed, Length: 1145, dtype: int64

Было принято решение разделить их по классам, в которых будет не менее 500 экземпляров на трейне, если встретим 0, то дадим им метку 0.

(Изначально я пытался использовать CountVectorizer и OneHotEncoder, но они не давали такого хорошего результата, скорее всего из-за того, что если мы в дереве решений сплитим, по какому-то из сгенерированных ими признаков, мы будем использовать слишком мало информации, например принадлежит ли он одной конкретной породе или есть ли в названии породы какое-то слово, кажется, что делить так не очень эффективно.)

In [30]:
breeds = df_train.Breed.value_counts().to_dict()

my_breeds_encoder = {}

current_sum = 0

for key, value in breeds.items():
    current_sum += value
    if current_sum > 500:
        current_sum = 0
        for prev in my_breeds_encoder.keys():
            my_breeds_encoder[prev] += 1
    my_breeds_encoder[key] = 1

my_breeds_encoder


{'Domestic Shorthair Mix': 20,
 'Pit Bull Mix': 19,
 'Chihuahua Shorthair Mix': 18,
 'Labrador Retriever Mix': 17,
 'Domestic Medium Hair Mix': 16,
 'German Shepherd Mix': 16,
 'Domestic Longhair Mix': 15,
 'Siamese Mix': 15,
 'Australian Cattle Dog Mix': 14,
 'Dachshund Mix': 14,
 'Boxer Mix': 14,
 'Miniature Poodle Mix': 13,
 'Border Collie Mix': 13,
 'Rat Terrier Mix': 13,
 'Catahoula Mix': 13,
 'Australian Shepherd Mix': 12,
 'Jack Russell Terrier Mix': 12,
 'Yorkshire Terrier Mix': 12,
 'Miniature Schnauzer Mix': 12,
 'Siberian Husky Mix': 12,
 'Chihuahua Longhair Mix': 12,
 'Domestic Shorthair': 11,
 'Beagle Mix': 11,
 'Rottweiler Mix': 11,
 'American Bulldog Mix': 11,
 'Pointer Mix': 11,
 'Shih Tzu Mix': 11,
 'Australian Kelpie Mix': 11,
 'Chihuahua Shorthair/Dachshund': 10,
 'American Staffordshire Terrier Mix': 10,
 'Staffordshire Mix': 10,
 'Cairn Terrier Mix': 10,
 'Chihuahua Shorthair': 10,
 'Great Pyrenees Mix': 10,
 'German Shepherd': 10,
 'Labrador Retriever/Pit Bull': 1

#### Пол

Пола, как мы видим, четыре: стерилизованные и нестерилизованные самки и самцы. Также пол может быть неизвестен.


In [31]:
df_train["SexuponOutcome"].value_counts()


Neutered Male    6802
Spayed Female    6223
Intact Female    2466
Intact Male      2438
Unknown           780
Name: SexuponOutcome, dtype: int64

#####  OrdinalEncoder

Здесь я уже воспользуюсь OneHotEncoder, потому что мы сгенерируем не очень много признаков, как в случае с породой.

In [32]:
ordinal_encoder = OrdinalEncoder(
    handle_unknown="use_encoded_value", unknown_value=-1)
ordinal_encoder.fit(df_train[["AnimalType", "Color"]])
ordinal_encoder.transform(df_train[["AnimalType", "Color"]])


array([[  0.,  36.],
       [  0., 270.],
       [  1., 320.],
       ...,
       [  1., 293.],
       [  1.,   4.],
       [  0.,   4.]])

### Единая матрица признаков

In [33]:
def prepare_features(df, animal_encoder=None, sex_encoder=None, color_encoder=None, scaler=None):

    columns_categorical = ["AnimalType", "SexuponOutcome", 'Breed']
    date_age_columns = ["DateTime", "AgeuponOutcome"]

    dt = pandas_dates2number(df["DateTime"])
    age = pandas_ages2number(df["AgeuponOutcome"])

    date_age_features = pd.concat([dt, age], axis=1)

    if animal_encoder is None and sex_encoder is None and color_encoder is None and scaler is None:
        animal_encoder = LabelEncoder()

        animal_encoder.fit(df["AnimalType"])

        sex_encoder = LabelEncoder()
        sex_encoder.fit(df["SexuponOutcome"])

        color_encoder = CountVectorizer()
        color_encoder.fit(df["Color"])
        
        scaler = StandardScaler()
        scaler.fit(date_age_features)
        
    date_age_features =  scaler.transform(date_age_features)
            
    animal_feature = animal_encoder.transform(df["AnimalType"])

    sex_feature = sex_encoder.transform(df["SexuponOutcome"])
    
    breed_feature = df["Breed"].apply(lambda x: my_breeds_encoder[x] if x in my_breeds_encoder.keys() else 0)

    color_features = color_encoder.transform(df["Color"])

    color_feature_names = color_encoder.get_feature_names()

    X = sparse.hstack([animal_feature[:, np.newaxis], sex_feature[:,
                      np.newaxis], breed_feature[:, np.newaxis], color_features,  date_age_features])

    return X, list(columns_categorical) + list(color_feature_names) + date_age_columns, animal_encoder, sex_encoder, color_encoder, scaler


In [34]:
X_train, fnames, animal_encoder, sex_encoder, color_encoder, scaler = prepare_features(df_train)
y_train = df_train["Outcome"]
X_test, _, _, _, _, _ = prepare_features(df_test, animal_encoder, sex_encoder, color_encoder, scaler)

X_train.shape, X_test.shape, np.array(fnames)


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\User\AppData\Local\Temp\ipykernel_17012\1833675630.py:38: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  np.newaxis], breed_feature[:, np.newaxis], color_features,  date_age_features])
c:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
C:\Users\User\AppData\Local\Temp\ipykernel_17012\1833675630.py:38: FutureWarning: Support for m

((18710, 41),
 (8019, 41),
 array(['AnimalType', 'SexuponOutcome', 'Breed', 'agouti', 'apricot',
        'black', 'blue', 'brindle', 'brown', 'buff', 'calico', 'chocolate',
        'cream', 'fawn', 'flame', 'gold', 'gray', 'lilac', 'liver', 'lynx',
        'merle', 'orange', 'pink', 'point', 'red', 'ruddy', 'sable',
        'seal', 'silver', 'smoke', 'tabby', 'tan', 'tick', 'tiger',
        'torbie', 'tortie', 'tricolor', 'white', 'yellow', 'DateTime',
        'AgeuponOutcome'], dtype='<U14'))

## Сетка параметров

Min_samples_leaf и min_samples_split очень сильно связаны, поэтому я решил, что разумно перебирать только один из них, я взял min_samples_split.

Очень естественно перебирать количество деревьев в лесе решений, поэтому мы перебираем с шагом 50 от 50 до 250, это кажется разумно.

Также перебираем глубину дерева, после пары запусков стало понятно, что оптимально значение явно не меньше 10, но и не очень большое, поэтому я перебираю от 14 до 32.

Технически конечно можно перебирать критерий, но у меня опять же на паре запусков стало видно, что критерий gini работает на порядок лучше, так что я решил воспользоваться рекомендацией sklearn.

In [35]:
param_grid = [
    {
        "min_samples_split": range(2, 7, 2),
        "max_depth": range(14, 34, 2),
        "n_estimators": [50, 100, 150, 200, 250]}]

param_grid


[{'min_samples_split': range(2, 7, 2),
  'max_depth': range(14, 34, 2),
  'n_estimators': [50, 100, 150, 200, 250]}]

## Обучение и предсказание

In [36]:
clf = GridSearchCV(RandomForestClassifier(random_state=0, class_weight="balanced", min_samples_leaf=2),
                    param_grid,
                    scoring=score,
                    verbose=1,
                    n_jobs=-1,
                    cv=StratifiedKFold(n_splits=FOLD_K, random_state=0, shuffle=True))

%time clf.fit(X_train, y_train)

print("Best params on dev set:")
print(clf.best_params_)
print(clf.best_score_)

print("Scores on development set:")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

best_estimator = clf.best_estimator_
best_estimator.fit(X_train, y_train)

y_pred = best_estimator.predict(X_test)
pd.DataFrame({"ID": df_test["ID"], "Outcome": y_pred}).to_csv(
    "submission_simple_" + score + ".csv", index=None)


Fitting 4 folds for each of 150 candidates, totalling 600 fits
Wall time: 12min 3s
Best params on dev set:
{'max_depth': 28, 'min_samples_split': 2, 'n_estimators': 200}
0.44255217077108777
Scores on development set:
0.429 (+/-0.008) for {'max_depth': 14, 'min_samples_split': 2, 'n_estimators': 50}
0.428 (+/-0.009) for {'max_depth': 14, 'min_samples_split': 2, 'n_estimators': 100}
0.430 (+/-0.003) for {'max_depth': 14, 'min_samples_split': 2, 'n_estimators': 150}
0.431 (+/-0.006) for {'max_depth': 14, 'min_samples_split': 2, 'n_estimators': 200}
0.429 (+/-0.005) for {'max_depth': 14, 'min_samples_split': 2, 'n_estimators': 250}
0.429 (+/-0.008) for {'max_depth': 14, 'min_samples_split': 4, 'n_estimators': 50}
0.428 (+/-0.009) for {'max_depth': 14, 'min_samples_split': 4, 'n_estimators': 100}
0.430 (+/-0.003) for {'max_depth': 14, 'min_samples_split': 4, 'n_estimators': 150}
0.431 (+/-0.006) for {'max_depth': 14, 'min_samples_split': 4, 'n_estimators': 200}
0.429 (+/-0.005) for {'max_de

Данное решение имеет Score: 0.43413 на Kaggle.

### Бонус: поверхностный анализ обученной модели

In [37]:
eli5.explain_weights(
    best_estimator, target_names=LABEL2OUTCOME, feature_names=fnames)


Weight,Feature
0.2641 ± 0.0532,AgeuponOutcome
0.2435 ± 0.0360,DateTime
0.1672 ± 0.0431,SexuponOutcome
0.1095 ± 0.0487,Breed
0.0432 ± 0.0574,AnimalType
0.0271 ± 0.0146,white
0.0191 ± 0.0112,black
0.0166 ± 0.0213,tabby
0.0161 ± 0.0110,brown
0.0107 ± 0.0068,blue
